In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [135]:
from dataclasses import dataclass, field
import typing as t
import itertools as it
import collections as c
import json
from copy import deepcopy
import math
import time
import functools as ft
import numpy as np
import tqdm.notebook

In [38]:
def parse_fl(fl):
    lns = open(fl).read().splitlines()
    grid = np.array([list(ln) for ln in lns])
    return grid, {(i,j) for i, ln in enumerate(lns) for j, c in enumerate(ln) if c=='#'}

In [53]:
small_grid, small_elves = parse_fl("small-test.txt")
test_grid, test_elves = parse_fl("test.txt")
input_grid, input_elves = parse_fl("input.txt")

In [154]:
small_grid

array([['.', '.', '#', '#', '.'],
       ['.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '.'],
       ['.', '.', '.', '#', '.'],
       ['.', '.', '#', '.', '.'],
       ['.', '.', '.', '.', '.']], dtype='<U1')

In [155]:
small_elves

{(0, 2), (0, 3), (2, 2), (3, 3), (4, 2)}

In [50]:
MV_ORDERS = [
    ['n', 's', 'w', 'e'],
    ['s', 'w', 'e', 'n'],
    ['w', 'e', 'n', 's'],
    ['e', 'n', 's', 'w'],
]

In [129]:
def next_pos(pos, elves, mv_order):
    if not is_other_elf_adj(pos=pos, elves=elves):
        return pos
    mv_map = {
        'n': {'adj_fn': _adj_north, 'pi': -1, 'pj': 0},
        's': {'adj_fn': _adj_south, 'pi': +1, 'pj': 0},
        'e': {'adj_fn': _adj_east, 'pi': 0, 'pj': +1},
        'w': {'adj_fn': _adj_west, 'pi': 0, 'pj': -1},
    }
    for mv in mv_order:
        mv_info = mv_map[mv]
        if mv_info['adj_fn'](pos) & elves:
            continue
        else:
            next_pos = (pos[0]+mv_info['pi'], pos[1]+mv_info['pj'])
#             print(mv)
            break
    else:
        next_pos = pos
    return next_pos


def _adj_north(pos):
    pi, pj = pos
    return {(pi-1, pj-1), (pi-1, pj), (pi-1, pj+1)}

def _adj_south(pos):
    pi, pj = pos
    return {(pi+1, pj-1), (pi+1, pj), (pi+1, pj+1)}

def _adj_east(pos):
    pi, pj = pos
    return {(pi-1, pj+1), (pi, pj+1), (pi+1, pj+1)}

def _adj_west(pos):
    pi, pj = pos
    return {(pi-1, pj-1), (pi, pj-1), (pi+1, pj-1)}
                

def is_other_elf_adj(pos, elves):
    return any(adj in elves for adj in _adj_poses(pos))

def _adj_poses(pos):
    return {(pos[0]+i, pos[1]+j) for i, j in it.product([-1,0,1], [-1,0,1]) if not i==j==0}

In [156]:
MV_ORDERS

[['n', 's', 'w', 'e'],
 ['s', 'w', 'e', 'n'],
 ['w', 'e', 'n', 's'],
 ['e', 'n', 's', 'w']]

In [152]:
def play(elves, max_rounds=1000_000_000_000, mv_orders_gvn=MV_ORDERS):
    elves = deepcopy(elves)
    _round = 1
    mv_orders = it.cycle(mv_orders_gvn)
    for _round in tqdm.notebook.tqdm(range(max_rounds)):
        is_finished, new_pos_map, pos_ctr = round1(elves=elves, mv_order=next(mv_orders))
        if is_finished:
            print(f"Finished at round: {_round+1}")
            break
        elves = round2(new_pos_map=new_pos_map, pos_ctr=pos_ctr)
    return num_empty_grid(elves)

def round1(elves, mv_order):
    is_finished, new_pos_map, pos_ctr = True, {}, c.Counter()
    for old_pos in elves:
        np = next_pos(pos=old_pos, elves=elves, mv_order=mv_order)
        is_finished &= old_pos == np
        new_pos_map[old_pos] = np
        pos_ctr[np] += 1
    return is_finished, new_pos_map, pos_ctr

def round2(new_pos_map, pos_ctr):
    return {
        new_pos if pos_ctr[new_pos] == 1 else old_pos
        for old_pos, new_pos in new_pos_map.items() 
    }

def num_empty_grid(elves):
    pis, pjs = list(zip(*sorted(elves)))
    return (
        ((max(pis)-min(pis)+1) * (max(pjs)-min(pjs)+1))
        - len(elves)
    )

In [153]:
play(elves=input_elves, mv_orders_gvn=MV_ORDERS)

  0%|          | 0/1000000000000 [00:00<?, ?it/s]

Finished at round: 938


15651